In [ ]:
from importlib import reload
from sklearn.model_selection import  TimeSeriesSplit,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import joblib
import pickle
from scipy.stats import uniform, randint,probplot,shapiro
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier as scikit_KerasClassifier
from scikeras.wrappers import KerasClassifier as scikeras_KerasClassifier
import pandas as pd
import numpy as np
import random as rd
from pathlib import Path
import matplotlib.pyplot as plt
import maria_import_export as maria
import model_mngr as modmgr
import split_merge as sm
import add_indicators as indic
import balance
reload(maria)
reload(indic)
reload(sm)
reload(modmgr)
reload(balance)


Load data for CW8 ETF from Maria
Add indicators and labels and save the dataset on disk

In [ ]:
PATH_DATA = "../../../Data/DTS_FULL/"
PATH_MODEL = "../../../Data/Models/"
PATH_RES = "../../../Data/Res/"
symb = "CW8"
dts_name = "DCA_CLOSE_1D_V1"
rnd_key = int(maria.get_conf("RANDOM_KEY"))
df = pd.DataFrame()
if "con" in locals():
    maria.close_connection(con)
con = maria.get_connection()
df = maria.get_candles_to_df(con=con, symbol=symb, only_close=True)
df = indic.add_indicators_to_df(con=con, df_in=df, dts_name=dts_name)
df.sort_index(inplace=True)
#df.round(5).to_csv(PATH_DATA+dts_name+"_full.zip", sep=",")


Clean the dataset droping useless features
Split the dataset by labels, train, val, conf part

In [ ]:
df = indic.drop_indicators_by_type(
    con=con, df_in=df, dts_name=dts_name, symbol=symb, ind_type=0)
list_label = indic.get_ind_list_by_type_for_dts(
    con=con, dts_name=dts_name, symbol=symb, ind_type=2)
print(list_label)
lab_studied = "lab_perf_21d"

df_clean=sm.remove_columns(df_in=df,df_labels=list_label,str_label=lab_studied)

selection of the df studied and plot data to check
!! change lab_studied and algo_studied before training !!

In [ ]:
# dict_split.keys()
algo_studied = "LSTM_CLASS"
lab_df_studied = "df_"+lab_studied
seq_len = 20
#categ_lab={0:[0.06,100],1:[0.04,0.06],2:[0.015,0.03],3:[0,0.015],4:[-0.03,0],5:[-100,-0.03]}#categ for CW8 lab 21d
#categ_lab={0:[0.05,100],1:[0.025,0.05],2:[0.01,0.025],3:[-0.01,0.01],4:[-0.04,-0.01],5:[-100,-0.04]}#categ for CW8 lab 21d
categ_lab={0:[0.04,100],1:[0.02,0.04],2:[0.00,0.02],3:[-0.025,0.00],4:[-100.0,-0.025]}#categ for CW8 lab 21d

df_class=balance.add_lab_by_class(df_in=df_clean,str_label=lab_studied,categ=categ_lab,bool_replace_label=True)

df_selected = indic.drop_indicators_not_selected(con=con, df_in=df_class, dts_name=dts_name, symbol=symb,label=lab_studied,algo=algo_studied)

# df_selected.round(5).to_csv(PATH_DATA+dts_name+"_train_colab_lstm_2307_1.zip", sep=",")

# dict_split = sm.split_df_by_label_strat(
#     df_in=df_class, list_label=list_label['LABEL'].tolist(), split_timeframe="M",random_split=False,split_strat=(70,15,15))

x_train, y_train, x_valid, y_valid, x_conf, y_conf = sm.split_df_strat_lstm(df_in=df_selected,str_label=lab_studied,split_strat=(70,15,15),sequence_length=seq_len)

#print(df_test.info())

#call function replace lab_studied by lab_studied_class .sort_index(inplace=True)
print(f"SHAPES : {df_class.shape=} {x_train.shape=}  {y_train.shape=}  {x_valid.shape=} {y_valid.shape=} {x_conf.shape=} {y_conf.shape=} ")

Check label

In [ ]:
data = df_selected[lab_studied]
data.hist(figsize=(8, 3),bins=50,)
print(data.value_counts().sort_index())

In [ ]:
#TomekLinks#RandomUnderSampler
# df_usamp=balance.class_undersampler(df_in=df_selected,str_label=lab_studied,str_method="RandomUnderSampler",dict_strat={0:34,1:82,2:300,3:300,4:300,5:151})
# df_usamp=balance.class_oversampler(df_in=df_usamp,str_label=lab_studied,str_method="SMOTE",dict_strat={0:100,1:200,2:300,3:300,4:300,5:300})
#df_usamp.info()
# data = df_usamp[lab_studied]
# data.hist(figsize=(8, 3),bins=50,)


Features analysis

In [ ]:
print(df_selected.describe())
(df_selected).hist(figsize=(12, 10))

In [ ]:
df_norm,norm_scaler= balance.normalize_df(df_in=df_selected,str_label=lab_studied,tuple_ft_range=(-1,1))
#data = df_selected['pos_sma200']
#df_norm['pos_sma200'].hist(figsize=(8, 3),bins=50,)
print(df_norm.describe())
(df_norm).hist(figsize=(12, 10))

x_train, y_train, x_valid, y_valid, x_conf, y_conf = sm.split_df_strat_lstm(df_in=df_norm,str_label=lab_studied,split_strat=(70,15,15),sequence_length=seq_len)

#print(df_test.info())

#call function replace lab_studied by lab_studied_class .sort_index(inplace=True)
print(f"SHAPES : {df_norm.shape=} {x_train.shape=}  {y_train.shape=}  {x_valid.shape=} {y_valid.shape=} {x_conf.shape=} {y_conf.shape=} ")

file_name=dts_name+"_train_colab_lstm_norm_2307_1"
df_norm.round(5).to_csv(PATH_DATA+file_name+".zip", sep=",")
joblib.dump(norm_scaler,filename=PATH_DATA+file_name+"_scaler.save")

In [ ]:
input_dim = x_train.shape[-1]
window_size = seq_len
dropout = 0.2
num_classes = 5

# cat_y_train = keras.utils.to_categorical(col_y_train, num_classes)
# cat_y_valid = keras.utils.to_categorical(col_y_valid, num_classes)

# df_x_train_exp = np.expand_dims(df_x_train, axis=2)
# df_x_valid_exp = np.expand_dims(df_x_valid, axis=2)


model_LSTM = Sequential()
model_LSTM.add(LSTM(units=128, return_sequences=True,
               input_shape=(window_size, input_dim)))
model_LSTM.add(Dropout(rate=dropout))
model_LSTM.add(Bidirectional(LSTM((window_size * 2), return_sequences=True)))
model_LSTM.add(Dropout(rate=dropout))
model_LSTM.add(Bidirectional(LSTM(window_size, return_sequences=False)))
model_LSTM.add(Dense(units=num_classes, activation='softmax'))

model_LSTM.compile(loss='categorical_crossentropy',
                   optimizer='adam', metrics=['accuracy'])

history = model_LSTM.fit(x_train, y_train, #batch_size=32,
                         shuffle=False, epochs=16, validation_data=(x_valid, y_valid),verbose=0)

train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot loss
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# TRAINING MODEL WITH SCIKIT LEARN
input_dim = x_train.shape[-1]
window_size = seq_len
num_classes = 5
rd = 28

print(f"{input_dim=}")
print(
    f"SHAPES : {x_train.shape=}  {y_train.shape=}  {x_valid.shape=} {y_valid.shape=} ")

lstm_model = scikit_KerasClassifier(build_fn=modmgr.create_sklearn_lstm_model,
                             input_dim=x_train.shape[-1], window_size=seq_len, num_classes=num_classes, layers=[64, seq_len * 2, seq_len], dropout=0.2)
es = EarlyStopping(monitor='val_loss', patience=5, mode='min', verbose=2)
# mc = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
history = lstm_model.fit(x_train, y_train, epochs=40, validation_data=(
    x_valid, y_valid), callbacks=[es], verbose=0)

# saved_model = load_model('best_model.h5')

train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot loss
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# TRAINING MODEL WITH SCIKERAS
input_dim = x_train.shape[-1]
window_size = seq_len
num_classes = 5
rd = 28
epochs=100
filename_tmp_model="tmp_best_model.h5"

print(f"{input_dim=}")
print(
    f"SHAPES : {x_train.shape=}  {y_train.shape=}  {x_valid.shape=} {y_valid.shape=} ")

es = EarlyStopping(monitor="val_loss", patience=10, mode="min", verbose=2)
mc = ModelCheckpoint(filename_tmp_model, monitor="val_loss", save_best_only=True)
lstm_model = scikeras_KerasClassifier(model=modmgr.create_scikeras_lstm_model, layers=[64, seq_len * 2, seq_len], dropout=0.2, callbacks=[es,mc])

history = lstm_model.fit(x_train, y_train, epochs=epochs, validation_data=(
    x_valid, y_valid), verbose=0)#/** param à revoir ***/

train_loss = history.history_['loss']
val_loss = history.history_['val_loss']

# Plot loss
epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo-', label='Training Loss')
plt.plot(epochs, val_loss, 'ro-', label='Validation Loss')
plt.legend()
plt.show()

saved_model = load_model(filename_tmp_model)
saved_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
loss, accuracy = saved_model.evaluate(x_valid, y_valid)
print(f"Save model validation Loss: {loss} Accuracy {accuracy}" )

In [ ]:
# RANDOM/GRID TRAINING MODEL WITH SCIKERAS
input_dim = x_train.shape[-1]
num_classes = 5
rd = 28
epochs = 50
n_iter = 2
filename_tmp_model = "tmp_best_model.h5"

es = EarlyStopping(monitor="accuracy", patience=3, mode="max", verbose=2)
mc = ModelCheckpoint(filename_tmp_model, monitor="accuracy",
                     mode="max", save_freq="epoch", save_best_only=True)
lstm_model = scikeras_KerasClassifier(model=modmgr.create_scikeras_lstm_model, optimizer="adam",
                                      optimizer__lr=0.1, model__layers=[64, seq_len * 2, seq_len], model__dropout=0.2, callbacks=[es, mc], verbose=0)

# parameters = {
#     'optimizer__lr': [0.05, 0.1],
#     'optimizer__momentum': [0.5, 0.7, 0.9],
#     'model__layers': [ [input_dim, 32], [input_dim, 16], [input_dim, 32, 16],
#                        [32, seq_len * 2], [32,
#                                                 seq_len], [32, seq_len * 2, seq_len],
#                        [64, seq_len * 2], [64,
#                                                 seq_len], [64, seq_len * 2, seq_len],
#                        [128, seq_len * 2], [128, seq_len], [128, seq_len * 2, seq_len],],
#     'model__dropout': [0.1, 0.2],
#     'fit__epochs': [epochs],
#     'fit__batch_size':[32,64,128,256]
# }

parameters = {
    'optimizer__lr': [ 0.1],
    'optimizer__momentum': [ 0.9],
    'model__layers': [ [32, seq_len * 2]],
    'model__dropout': [0.1],
    'fit__epochs': [epochs],
    'fit__batch_size':[32]
}

# Object X_SearchCV
time_split = TimeSeriesSplit(n_splits=3)
search = RandomizedSearchCV(estimator=lstm_model, param_distributions=parameters,
                                 n_jobs=1, scoring='accuracy', cv=time_split, n_iter=n_iter, random_state=rd, verbose=4,error_score='raise')

search_result = search.fit(x_train, y_train)

# print(f"Best Score: {search_result.best_score_}")
# print(f"Best Params: {search_result.best_params_}")

results_df = pd.DataFrame(search_result.cv_results_)
top_10_results = results_df.sort_values('mean_test_score', ascending=False).head(10)

for index, row in top_10_results.iterrows():
    print(f"Hyperparameters: {row['params']}")
    print(f"Mean Score: {row['mean_test_score']}, Std Score: {row['std_test_score']}")


In [ ]:
# test du model sur la validation
saved_model = load_model(filename_tmp_model)
saved_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
loss, accuracy = saved_model.evaluate(x_valid, y_valid)

print(f"Validation Loss: {loss} Accuracy :{accuracy}")

save model

In [ ]:
ext_pkl = ".pkl"
suffix = "_lstm_v1"
joblib.dump(model_LSTM, PATH_MODEL+symb+"_"+lab_studied+suffix+ext_pkl)

Generate scores values  for the dataset and save in csv

In [ ]:
# dict_split.keys()
algo_studied = "LSTM_CLASS"
seq_len = 20
ext_pkl = ".pkl"
suffix = "_lstm_v1"

x_eval_lstm, y_eval_lstm,df_predict = sm.prepare_sequences_with_df(df_in=df_selected,str_label=lab_studied,sequence_length=seq_len)

model_LSTM_check = joblib.load(PATH_MODEL+symb+"_"+lab_studied+suffix+ext_pkl)
arr_res = model_LSTM_check.predict(x_eval_lstm)

print(f"{df_predict.shape=} {x_eval_lstm.shape=} {arr_res.shape=} ")

df_predict["y_eval"] = np.concatenate([np.array([np.nan] * (df_predict.shape[0] - len(y_eval_lstm))), np.argmax(y_eval_lstm, axis=1)]) 
df_predict["predict"] = np.concatenate([np.array([np.nan] * (df_predict.shape[0] - len(arr_res))), np.argmax(arr_res, axis=1)]) 

df_predict.describe()

In [ ]:
df_predict.dropna(inplace=True)
df_predict.round(5).to_csv(
    PATH_RES+dts_name+"_"+lab_studied+"_predict_"+algo_studied+".zip", sep=",")

PREPARATION FOR BACKTEST

In [ ]:
df_can = maria.get_candles_to_df(con=con, symbol=symb, only_close=False)
df_can.sort_index(inplace=True)
df_can.describe()

In [ ]:
df_bt=sm.join_dataframes_backtest(df_candle=df_can,df_score=df_prep,str_col_score='predict_ridge')
df_bt.describe(include='all')
df_bt.round(5).to_csv(
     "../../../Data/backtest/"+dts_name+"_"+lab_studied+"_predict_BT.zip", sep=",")

In [ ]:
import numpy as np
from sklearn.datasets import make_classification
from tensorflow import keras

from scikeras.wrappers import KerasClassifier


X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

def get_model(hidden_layer_dim, meta):
    # note that meta is a special argument that will be
    # handed a dict containing input metadata
    n_features_in_ = meta["n_features_in_"]
    X_shape_ = meta["X_shape_"]
    n_classes_ = meta["n_classes_"]
    print(f"{n_features_in_=}")

    model = keras.models.Sequential()
    model.add(keras.layers.Dense(n_features_in_, input_shape=X_shape_[1:]))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(hidden_layer_dim))
    model.add(keras.layers.Activation("relu"))
    model.add(keras.layers.Dense(n_classes_))
    model.add(keras.layers.Activation("softmax"))
    return model

clf = KerasClassifier(
    get_model,
    loss="sparse_categorical_crossentropy",
    hidden_layer_dim=100,
)

clf.fit(X, y)
y_proba = clf.predict_proba(X)